In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from skimage.io import imread, imshow

In [2]:
YOLO_MODELS_PATH = "yolo_models/"
TRAIN_PATH = '../../data/raw_data/malaria/transformed_images/'

MASK_CATEGORIES = ['trophozoite','ring', 'schizont', 'gametocyte']
STR_MASK_CATEGORIES = '_'.join(MASK_CATEGORIES)

In [3]:
# generate different colors for different classes 
with open(YOLO_MODELS_PATH+"obj.names", 'r') as f:
#with open(YOLO_MODELS_PATH+"coco.names", 'r') as f:
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 4))

In [ ]:
net = cv2.dnn.readNetFromDarknet(YOLO_MODELS_PATH+"YOLOv2-malaria_201908120022_trophozoite_ring_schizont_gametocyte_416_416.cfg", \
                                YOLO_MODELS_PATH + "YOLOv2-malaria_201908120022_trophozoite_ring_schizont_gametocyte_416_416_best.weights")

In [4]:
net = cv2.dnn.readNetFromDarknet(YOLO_MODELS_PATH+"YOLO-malaria_201908112336_trophozoite_ring_schizont_gametocyte_800_800.cfg", \
                                YOLO_MODELS_PATH + "YOLO-malaria_201908112336_trophozoite_ring_schizont_gametocyte_800_800_best.weights")

In [ ]:
net = cv2.dnn.readNetFromDarknet(YOLO_MODELS_PATH+"yolo.cfg", \
                                YOLO_MODELS_PATH + "yolo.weights")

In [ ]:
net = cv2.dnn.readNet(YOLO_MODELS_PATH + "YOLO-malaria_201908112336_trophozoite_ring_schizont_gametocyte_800_800_best.weights", \
                      YOLO_MODELS_PATH+"YOLO-malaria_201908112336_trophozoite_ring_schizont_gametocyte_800_800.cfg")

In [5]:
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [8]:
net

<dnn_Net 0x7fca31076110>

### Read Image

In [6]:
test_ids = np.load(TRAIN_PATH+'malaria_test_transformed_256_256_names.npy',allow_pickle=True)


In [ ]:
n = 8
img = imread(test_ids[n])
plt.imshow(img)

In [ ]:
img = np.squeeze(Y_test[n])
plt.imshow(img)

In [ ]:
# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    print(class_id)
    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
#image = cv2.imread(test_ids[n])
image = cv2.imread("dog.jpg")
Width = image.shape[1]
Height = image.shape[0]
scale = 1/255


blob = cv2.dnn.blobFromImage(image, scale, (800,800), (0,0,0), True, crop=False)

In [ ]:
layer_names = net.getLayerNames()
    
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
# set input blob for the network
net.setInput(blob)

In [ ]:
outs = net.forward(output_layers)

In [ ]:
#outs

In [ ]:
train_ids = np.load(TRAIN_PATH+'malaria_train_transformed_256_256_names.npy',allow_pickle=True)

In [7]:
#image = cv2.imread(test_ids[n])
count =0
tested = 0
for im in test_ids:
    tested += 1
    image = cv2.imread(im)
    Width = image.shape[1]
    Height = image.shape[0]
    scale = 1/255


    blob = cv2.dnn.blobFromImage(image, scale, (800,800),  [0,0,0],swapRB=True, crop=False)
    
    net.setInput(blob)
    
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    outs = net.forward(output_layers)
    
    # initialization
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                count +=1
print(count)
print(tested)

0
0.39130923
0
0.49590388
1
0.4016983
0
0.43913156
0
0.5296549
0
0.36553296
1
0.36258245
1
0.42082244
1
0.36910337
0
0.47196642
0
0.6046126
0
0.31017798
0
0.421465
13
120


In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

# display output image    
cv2.imshow("object detection", image)

# wait until any key is pressed
cv2.waitKey()
    
 # save output image to disk
cv2.imwrite("object-detection.jpg", image)

# release resources
cv2.destroyAllWindows()

In [ ]:
class_ids